## Use Sci-Kit Allel

    this makes using calling and accessing data though vcf files easy as well as helps is visualising data
    

In [34]:
import gzip
import umap

with gzip.open('ALL.wgs.nhgri_coriell_affy_6.20140825.genotypes_has_ped.vcf.gz', mode='r') as f:
    lines=f.readlines(429100)

    
with gzip.open('sample.vcf.gz', mode='w') as f:
    for i in lines:
        f.write(i)
    f.close()
    
with gzip.open('sample.vcf.gz', mode='r') as f:
    lines=f.readlines(129100)
for i in lines:
    print(i)

b'##fileformat=VCFv4.1\n'
b'##FORMAT=<ID=GT,Number=1,Type=String,Description="Genotype">\n'
b'##GATKCommandLine=<ID=CombineVariants,Version=3.2-2-gec30cee,Date="Sun Sep 21 20:02:54 PDT 2014",Epoch=1411354974729,CommandLineOptions="analysis_type=CombineVariants input_file=[] showFullBamList=false read_buffer_size=null phone_home=AWS gatk_key=null tag=NA read_filter=[] intervals=null excludeIntervals=null interval_set_rule=UNION interval_merging=ALL interval_padding=0 reference_sequence=/home/armartin/bustamante/genomes/hg19/1KG_v37/human_g1k_v37.fa nonDeterministicRandomSeed=false disableDithering=false maxRuntime=-1 maxRuntimeUnits=MINUTES downsampling_type=BY_SAMPLE downsample_to_fraction=null downsample_to_coverage=1000 baq=OFF baqGapOpenPenalty=40.0 refactor_NDN_cigar_string=false fix_misencoded_quality_scores=false allow_potentially_misencoded_quality_scores=false useOriginalQualities=false defaultBaseQualities=-1 performanceLog=null BQSR=null quantize_quals=0 disable_indel_quals=f

In [35]:
import allel
from collections import defaultdict

In [36]:
vcf = allel.read_vcf('sample.vcf.gz')


#internal memory structure , np array hence can do all operations of a numpy array as well as specific operations

#convert to biallelic variants

geno_array = allel.GenotypeArray(vcf['calldata/GT'])

print(vcf['samples'])

['HG00096' 'HG00097' 'HG00098' ... 'NA21142' 'NA21143' 'NA21144']


In [37]:
population_file = 'affy_samples.20141118.panel'

population_by_individual = defaultdict(int)
individuals_by_population = defaultdict(list)  # A dictionary containing all the individuals in a given population

for line in open(population_file,'r'):
    split_line = line.split()
    if split_line[0] == 'sample':  # header line
        continue

    sample_name = split_line[0]
    population_name = split_line[1]
    population_by_individual[sample_name] = population_name
    individuals_by_population[population_name].append(sample_name) 

populations = list(individuals_by_population.keys())

print(populations)
print(len(populations))

['GBR', 'FIN', 'CHS', 'PUR', 'CDX', 'CLM', 'IBS', 'PEL', 'PJL', 'KHV', 'ACB', 'GWD', 'ESN', 'BEB', 'MSL', 'STU', 'ITU', 'CEU', 'YRI', 'CHB', 'JPT', 'LWK', 'ASW', 'MXL', 'TSI', 'GIH']
26


In [38]:
data = geno_array.to_n_alt()

In [40]:
geno_array.T

array([[[ 0,  0,  0, ...,  0,  1,  0],
        [ 0, -1,  0, ...,  0,  1,  0],
        [ 0,  0,  0, ...,  0,  1,  0],
        ...,
        [ 0,  0,  0, ...,  0,  1,  0],
        [ 0,  0,  0, ...,  0,  0,  0],
        [ 0,  0,  0, ...,  0,  0,  0]],

       [[ 0,  0,  0, ...,  0,  1,  0],
        [ 0, -1,  0, ...,  0,  1,  0],
        [ 0,  0,  0, ...,  0,  1,  1],
        ...,
        [ 0,  0,  0, ...,  0,  1,  1],
        [ 0,  0,  1, ...,  0,  1,  1],
        [ 0,  0,  1, ...,  0,  1,  0]]], dtype=int8)

In [22]:
geno_array

<GenotypeArray shape=(29, 3450, 2) dtype=int8>
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
0/0 ./. 0/0 0/0 0/0 ... 0/0 ./. 0/0 0/0 0/0
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/1 0/0 0/1 0/1
...
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
1/1 1/1 1/1 1/1 1/1 ... 1/1 1/1 1/1 0/1 0/1
0/0 0/0 0/1 0/0 0/0 ... 0/1 0/1 0/1 0/1 0/0

In [23]:
arr = [0,1,2,4,5,6]

for i in arr:
    print(vcf['variants/ID'][i])
    print(vcf["variants/CHROM"][i],vcf["variants/POS"][i])
    print(allel.GenotypeArray(vcf["calldata/GT"])[i])

rs10458597
1 564621
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0
.
1 564773
0/0 ./. 0/0 0/0 0/0 ... 0/0 ./. 0/0 0/0 0/0
rs12565286
1 721290
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/1 0/0 0/1 0/1
rs3094315
1 752566
0/1 1/1 0/1 0/1 1/1 ... 1/1 0/1 0/1 0/1 0/0
rs2286139
1 761732
0/1 1/1 0/1 0/1 1/1 ... 1/1 1/1 0/1 1/1 0/1
rs11240776
1 765269
0/0 0/0 0/0 0/0 0/0 ... 0/0 0/0 0/0 0/0 0/0


In [24]:
len(proj_umap_pca[:,0])

22

In [25]:
pca_full = umap.UMAP().fit_transform(data)
proj_umap_gt = umap.UMAP(n_components=2, n_neighbors=15, min_dist=0.5).fit_transform(data)

In [26]:
proj_pca = umap.UMAP().fit_transform(data)
proj_umap_pca = umap.UMAP(n_components=3, n_neighbors=4, min_dist=0.5).fit_transform(proj_pca[:,:2])

/Users/vanisingh/anaconda3/lib/python3.6/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 2 separate connected components using meta-embedding (experimental)
  n_components


In [32]:
import matplotlib.pyplot as plt
import seaborn as sns



cmap = sns.cubehelix_palette(n_colors=26, start=0, rot=0.4, gamma=1.0, hue=0.8, light=0.85, dark=0.15, reverse=False, as_cmap=False)
sns.scatterplot(proj_umap_pca[:,0],proj_umap_pca[:,1],hue=populations,palette=cmap)

#sns.show()

ValueError: arrays must all be same length

In [14]:
len(set(populations[:21]))

21